In [128]:
import requests
from bs4 import BeautifulSoup
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp
from urllib.parse import urlparse, urljoin, urlsplit

In [129]:
print(urlsplit("http://bit.edu.cn"))
print(urlparse("http://bit.edu.cn"))

SplitResult(scheme='http', netloc='bit.edu.cn', path='', query='', fragment='')
ParseResult(scheme='http', netloc='bit.edu.cn', path='', params='', query='', fragment='')


In [130]:
# Set the main website to crawl
BASE_URL = 'http://bit.edu.cn'

# Set the maximum number of pages to crawl (optional)
MAX_PAGES = 1000

# Initialize the set of visited pages
visited_pages = set()

# Initialize the queue of pages to crawl
pages_to_crawl = [BASE_URL]

# Create a session object for making HTTP requests
session = requests.Session()

In [131]:
# Define a function to fetch a page asynchronously
async def fetch_page(url,session):
    async with aiohttp.ClientSession() as client_session:
        async with client_session.get(url) as response:
            return await response.text()

In [132]:
# # Define a function to crawl a page and extract its links
# async def crawl_page(url, session):
#     print(f'Crawling {url}...')
#     try:
#         page_html = await fetch_page(url, session)
#         soup = BeautifulSoup(page_html, 'html.parser')
#         links = soup.find_all('a')
#         for link in links:
#             link_url = link.get('href')
#             if link_url and 'http' not in link_url:
#                 abs_url = urljoin(url, link_url)
#                 domain = urlparse(BASE_URL).netloc
#                 if domain in abs_url and abs_url not in visited_pages:
#                     visited_pages.add(abs_url)
#                     pages_to_crawl.append(abs_url)
#     except Exception as e:
#         print(f'Error crawling {url}: {e}')

In [133]:
async def crawl_page(url, session):
    print(f'Crawling {url}...')
    try:
        page_html = await fetch_page(url, session)
        soup = BeautifulSoup(page_html, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            link_url = link.get('href')
            if link_url:
                if 'http' not in link_url:
                    abs_url = urljoin(url, link_url)
                else:
                    abs_url = link_url
                domain = urlparse(BASE_URL).netloc
                split_url = urlsplit(abs_url)
                if domain in split_url.netloc and abs_url not in visited_pages:
                    visited_pages.add(abs_url)
                    pages_to_crawl.append(abs_url)
    except Exception as e:
        print(f'Error crawling {url}: {e}')


In [134]:
# Define a function to run the crawler asynchronously
async def run_crawler():
    while len(pages_to_crawl) > 0 and len(visited_pages) < MAX_PAGES:
        url = pages_to_crawl.pop(0)
        if url not in visited_pages:
            visited_pages.add(url)
            await crawl_page(url, session)


In [135]:
# Run the crawler asynchronously
loop = asyncio.get_event_loop()
loop.run_until_complete(run_crawler())

# Print the list of visited pages
print(f'Visited {len(visited_pages)} pages:')
for page in visited_pages:
    print(page)

Crawling http://bit.edu.cn...
Visited 128 pages:
http://www.bit.edu.cn/ldxx/
https://www.bit.edu.cn/rcpy
http://ef.bit.edu.cn/
http://admission.bit.edu.cn/
http://bit.edu.cn/xww/xzw/xsjl1/cef06b72ec9849adad48e2438ca148de.htm
http://job.bit.edu.cn
http://www.bit.edu.cn/xwwjb2/
http://www.bit.edu.cn/xww/ztbd/dfxdr/
http://renshichu.bit.edu.cn/gbszdw/gblyys/index.htm
http://bit.edu.cn/gjjl_sjb/index.htm
http://grd.bit.edu.cn/
http://bit.edu.cn/xww/index.htm
http://bit.edu.cn/xww/xzw/xsjl1/b0cf811a20d74d96ba08919e2c2ce5c9.htm
http://bit.edu.cn/xww/xzw/xsjl1/750bcd56cf6448678d0138a969bdb982.htm
http://abroad.bit.edu.cn/
http://kjc.bit.edu.cn/kjpt/gjjzdsyszx/index.htm
http://bit.edu.cn/xww/xzw/xsjl1/6d7eb16080d144529d26cef32fceb048.htm
https://www.bit.edu.cn/xww/lgxb21/b228028794bc4db39bb22673ed99acf3.htm
http://bit.edu.cn/xww/wwhd/index.htm
http://bit.edu.cn/xww/gbmtlg/mtbl/239d1ddc3ff24c6b8711ee8bc8742cff.htm
http://sce.bit.edu.cn/cms/
https://www.bit.edu.cn/tkpt/
http://bit.edu.cn/xww/jxk